<a href="https://colab.research.google.com/github/tszalama/ml_rockpaperscissors/blob/main/PyTorch_rock%2C_paper%2C_scisors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
torch.manual_seed(42)
import torch.utils.data as data

Model definition

In [9]:
class Model(nn.Module):
  def __init__(self, num_inputs, num_outputs):
      super().__init__()
      # Initialize the first layer
      self.linear = nn.Linear(num_inputs, 3)
      # Initialize the second layer
      self.linear2 = nn.Linear(3, num_outputs)

      # Activation functions
      self.ReLU = nn.ReLU()
      self.Sigmoid = nn.Sigmoid()

  def forward(self, x):
      # Reshape the array to a format where both players' moves are encoded in a single 6-float value array
      x = x.reshape(-1, 6)
      # Apply layers and activation functions
      x = self.linear(x)
      x = self.ReLU(x)
      x = self.linear2(x)
      x = self.Sigmoid(x)
      return x

Data loader

In [7]:
class RPSDataset(data.Dataset):
    def __init__(self):
      super().__init__()
      # Results in the format [[player 1 result, player 2 result]]
      results = [[1, 1], [0, 1], [1, 0], [0, 1], [1, 1], [1, 0], [0, 1], [1, 0], [1, 1]]
      # Input data for player 1 and 2 in the format [[[Rock, Scissors, Paper], [Rock, Scissors, Paper]]]
      input_data = [  # expected results:
          [[0, 0, 1], [0, 0, 1]],  # 1, 1
          [[0, 0, 1], [0, 1, 0]],  # 0, 1
          [[0, 0, 1], [1, 0, 0]],  # 1, 0
          [[0, 1, 0], [0, 0, 1]],  # 0, 1
          [[0, 1, 0], [0, 1, 0]],  # 1, 1
          [[0, 1, 0], [1, 0, 0]],  # 1, 0
          [[1, 0, 0], [0, 0, 1]],  # 0, 1
          [[1, 0, 0], [0, 1, 0]],  # 1, 0
          [[1, 0, 0], [1, 0, 0]]   # 1, 1
      ]

      self.input_data = torch.tensor(input_data, dtype=torch.float32)
      self.results = torch.tensor(results, dtype=torch.float32)

    def __len__(self):
        return 9

    def __getitem__(self, idx):
        data_point = self.input_data[idx]
        data_label = self.results[idx]
        return data_point, data_label

Initialize classes

In [10]:
dataset = RPSDataset()
data_loader = data.DataLoader(dataset, batch_size=1, shuffle=True)

model = Model(6,2)

optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

lossfn = nn.MSELoss()

Training loop

In [12]:
for epoch in range(2000):
  for data_inputs, data_labels in data_loader:
    result = model(data_inputs)
    result = result.squeeze(dim=1) #reshape tensor
    
    loss = lossfn(result, data_labels)
    
    # backpropagation
    optimizer.zero_grad() 
    loss.backward()
    
    # adjust params
    optimizer.step()

In [15]:
print("\nResults after training:\n")
with torch.no_grad():
  for data_inputs, data_labels in data_loader:
    print(f'Expected: {data_labels.tolist()}, Result: {torch.round(model(data_inputs),decimals=0).tolist()}, Losss:{lossfn(model(data_inputs).squeeze(dim=1),data_labels).tolist()}\n')


Results after training:

Expected: [[0.0, 1.0]], Result: [[0.0, 1.0]], Losss:0.000943805614951998

Expected: [[1.0, 1.0]], Result: [[1.0, 1.0]], Losss:0.001756343524903059

Expected: [[0.0, 1.0]], Result: [[0.0, 1.0]], Losss:0.0007163074915297329

Expected: [[1.0, 1.0]], Result: [[1.0, 1.0]], Losss:0.0003866813494823873

Expected: [[1.0, 1.0]], Result: [[1.0, 1.0]], Losss:0.016681300476193428

Expected: [[1.0, 0.0]], Result: [[1.0, 0.0]], Losss:0.003906365483999252

Expected: [[1.0, 0.0]], Result: [[1.0, 0.0]], Losss:0.0014436400961130857

Expected: [[0.0, 1.0]], Result: [[0.0, 1.0]], Losss:0.009243005886673927

Expected: [[1.0, 0.0]], Result: [[1.0, 0.0]], Losss:1.371883899992099e-05

